# EXO 1


Nous allons expérimenter quelques approches "traditionnelles" de la classification des textes. Ces approches sont antérieures à la révolution de l'apprentissage en profondeur dans le traitement du langage naturel, mais sont souvent des moyens rapides et efficaces de former un classificateur de texte.


Pour nos données, nous allons travailler avec l'ensemble de données 20 Newsgroups, une collection classique de documents texte qui est souvent utilisée comme référence pour les modèles de classification de texte. L'ensemble contient des textes sur divers sujets, allant de l'informatique à la religion. Certains sujets sont étroitement liés les uns aux autres (tels que « Matériel IBM PC » et « Matériel Mac »), tandis que d'autres sont très différents (tels que « religion » ou « hockey »). Les 20 Newsgroups sont livrés avec la bibliothèque d'apprentissage automatique Scikit-learn, notre principal outil pour cet exercice. Il a été divisé en un ensemble de formation de 11 314 textes et un ensemble de test de 7 532 textes.


Pour cette fois on vous laisse utiliser vos connaisances pour faire l'ensemble des traitements que vous jugerai nécessaire pour un bon classifieur. 

## **Installation de librairies**

In [49]:
!pip install transformers
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **1. Chargement des bases de données**

In [50]:
from sklearn.datasets import fetch_20newsgroups

news_ds_train = fetch_20newsgroups(subset = 'train', shuffle = True, random_state = 42)
from pprint import pprint
pprint(list(news_ds_train.target_names))
#pprint(list(news_ds_train.data))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


Dans le bloc de code suivant, plusieurs éléments sont affichés :
* La liste des topics abordés dans le jeu de données
* Le nombre de news 
* Le contenu de la 1ère news
* Le topic de la 1ère news

In [51]:
print("Liste des topics")
print(news_ds_train.target_names)
print("\nNombre de news")
print(len(news_ds_train.data))
print("\nAffichage de la 1ère news")
print(news_ds_train.data[0])
print("\nTopic de la 1ère news")
print(news_ds_train.target_names[news_ds_train.target[0]])

Liste des topics
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

Nombre de news
11314

Affichage de la 1ère news
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of produc

# **2. Classification de textes**

## 2.1. Classification avec sklearn
Dans cette section, nous allons entrainer un modèle pour classifier les news du jeu de données 20newsgroup avec la librairie sklearn.

On commence par importer le jeu de données
* alt.atheism
* comp.graphics
* rec.motorcycles
* sci.crypt
* sci.med

Nous allons commencer à compter le nombre d'occurence des mots dans le corpus que nous venons de créer (ce corpus correspond à la variable *news_ds_train.data*).

Puis nous allons ensuite Vectoriser  ce corpus avec TF-IDF.

Ainsi nous enregistrerons  ces features du corpus dans la variable *corpus_X*.

In [52]:
categories = ['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt','sci.med']
news_ds_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)
print(news_ds_train.target_names)

categories = ['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt','sci.med']
news_ds_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)

['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.crypt', 'sci.med']


In [53]:
import sklearn.feature_extraction.text as skl_text

vectorizer = skl_text.TfidfVectorizer()
corpus_X = vectorizer.fit_transform(news_ds_train.data).toarray()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(corpus_X, news_ds_train.target, test_size = 0.25, random_state = 0)

print(vectorizer.get_feature_names_out())
print(len(vectorizer.get_feature_names_out()))

['00' '000' '0000' ... 'érale' 'ête' 'íålittin']
42672


# **Model Multinomiale Naive Bayes**

Nous allons maintenant entrainer un modèle appelé *Multinomial Naive Bayes* sur ce corpus. Les labels du corpus se trouvent dans la variable *news_ds_train.target*.

In [54]:
#Commencer par importer le module Multinomiale
from sklearn.naive_bayes import MultinomialNB
#Instancier le model
model_Multinomial = MultinomialNB()
#Entrianner le model
model_Multinomial.fit(X_train, y_train)
#Prediction
prediction = model_Multinomial.predict(X_test)
#Calculer le score 
from sklearn.metrics import accuracy_score
precision = accuracy_score(y_test, prediction)*100
print('La precison du score est', precision)



La precison du score est 96.21318373071529


Dans le bloc de code suivant, nous utilisons notre modèle entrainé sur des phrases pré-sélectionnées. Est-ce que votre modèle semble performant par rapport aux thématiques choisies ?

In [55]:
docs_new = ['Your post is based on the premise that the laws as they stand do not discriminate anybody', 'OpenGL on the GPU is fast']

# Vectorisons avec TF-IDF ces phrases
X_new_tfidf = vectorizer.fit_transform(docs_new)

predicted = model_Multinomial.predict(corpus_X)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, news_ds_train.target_names[category]))

'Your post is based on the premise that the laws as they stand do not discriminate anybody' => sci.med
'OpenGL on the GPU is fast' => comp.graphics


# **Avec le model de l'arbre de decision**

In [56]:
from sklearn.tree import DecisionTreeClassifier
#instanciation
model_DT = DecisionTreeClassifier()
#training
model_DT.fit(X_train, y_train)
#Prediction
prediction = model_DT.predict(X_test)
#Calculer le score
from sklearn.metrics import accuracy_score
precision = accuracy_score(y_test, prediction)*100
print('La precison du score est', precision)


La precison du score est 79.38288920056101


Essayons de faire une predictionn des phrases sur le model entrainé

In [57]:
docs_new = ['Your post is based on the premise that the laws as they stand do not discriminate anybody', 'OpenGL on the GPU is fast']

# Vectorisons avec TF-IDF ces phrases
X_new_tfidf = vectorizer.fit_transform(docs_new)

predicted = model_DT.predict(corpus_X)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, news_ds_train.target_names[category]))

'Your post is based on the premise that the laws as they stand do not discriminate anybody' => sci.med
'OpenGL on the GPU is fast' => comp.graphics
